### Basic Testing for Issue 129

In [1]:
import torch
import re
from functools import partial, partialmethod

In [2]:
def map_tuple(service, args, func):
    if service:
        return tuple(func(service, x) for x in args)
    else:
        return tuple(func(x) for x in args)

def map_dict(service, kwargs, func):
    if service:
        return {key:func(service, val) for key, val in kwargs.items()}
    else:
        return {key:func(val) for key, val in kwargs.items()}

def passer(func, *args, **kwargs):
    return partial(func, *args, **kwargs)

def passer_m(func, *args, **kwargs):
    return partialmethod(func, *args, **kwargs)

def compile_command(partial_func, has_self):
    func = partial_func.func
    args = partial_func.args
    kwargs = partial_func.keywords
    command = {}
    command['has_self'] = has_self
    if has_self:
        command['self'] = replace_tensorvar(args[0])
        args = args[1:]
    command['command'] = func.__name__
    command['args'] = map_tuple(None, args, replace_tensorvar)
    command['kwargs'] = map_dict(None, kwargs, replace_tensorvar)
    return command

In [3]:
class DummyWorker(object):
    def __init__(self):
        self.objects = {}

class DummyService(object):
    def __init__(self, worker):
        super().__init__()
        self.worker = worker
        
def replace_tensorvar(x):
    try:
        return '_fl.{}'.format(x.id) if (torch.is_tensor(x) 
            or isinstance(x, torch.autograd.Variable)) else [replace_tensorvar(i) for i in x]
    except (AttributeError, TypeError):
        return x
    
def retrieve_tensor(self, x):
    try:
        return self.worker.objects[id_tensorvar(x)]
    except TypeError:
        try:
            return [self.worker.objects[i] for i in id_tensorvar(x)]
        except TypeError:
            return x
    except KeyError:
        return x

def command_guard(command, allowed):
    if command not in allowed:
        raise RuntimeError(
            'Command "{}" is not a supported Torch operation.'.format(command))
    return command

def id_tensorvar(x):
    pat = re.compile('_fl.(.*)')
    try:
        if isinstance(x, str):
            return pat.search(x).group(1)
        else:
            return [id_tensorvar(i) for i in x]
    except AttributeError:
        return x
    
def process_command(self, command_msg):
    args = map_tuple(self, command_msg['args'], retrieve_tensor)
    kwargs = map_dict(self, command_msg['kwargs'], retrieve_tensor)
    has_self = command_msg['has_self']
    if has_self:
        command = command_guard(command_msg['command'],
            self.tensorvar_methods)
        obj_self = retrieve_tensor(self, command_msg['self'])
        command = eval('obj_self.{}'.format(command))
    else:
        command = command_guard(command_msg['command'], self.torch_funcs)
        command = eval('torch.{}'.format(command))
    return command(*args, **kwargs)

In [4]:
worker = DummyWorker()
self = DummyService(worker)

In [5]:
self.tensor_types = [torch.FloatTensor,
    torch.DoubleTensor,
    torch.HalfTensor,
    torch.ByteTensor,
    torch.CharTensor,
    torch.ShortTensor,
    torch.IntTensor,
    torch.LongTensor]

self.var_types = [torch.autograd.variable.Variable]

self.tensorvar_types = self.tensor_types + self.var_types

self.torch_funcs = dir(torch)

self.tensorvar_methods = list(
            set(
                [method
                    for tensorvar in self.tensorvar_types
                    for method in dir(tensorvar)]
                )
            )

In [6]:
x = torch.FloatTensor(3,3).uniform_()
x.id = 'poof'
y = torch.FloatTensor(3,3).uniform_()
y.id = 'boop'

In [7]:
self.worker.objects[x.id] = x
self.worker.objects[y.id] = y

In [8]:
part = passer(torch.add,x,other=y)
comm = compile_command(part, has_self=False)
part_m = passer_m(torch.FloatTensor.add, x, other=y)
comm_m = compile_command(part_m, has_self=True)
part_cat = passer(torch.cat,[x,y],dim=0)
comm_cat = compile_command(part_cat, has_self=False)

In [9]:
comm

{'args': ('_fl.poof',),
 'command': 'add',
 'has_self': False,
 'kwargs': {'other': '_fl.boop'}}

In [10]:
comm_m

{'args': (),
 'command': 'add',
 'has_self': True,
 'kwargs': {'other': '_fl.boop'},
 'self': '_fl.poof'}

In [11]:
comm_cat

{'args': (['_fl.poof', '_fl.boop'],),
 'command': 'cat',
 'has_self': False,
 'kwargs': {'dim': 0}}

In [12]:
print(x)


 0.0490  0.2131  0.9104
 0.3258  0.9905  0.4065
 0.5038  0.3850  0.3989
[torch.FloatTensor of size 3x3]



In [13]:
print(y)


 0.3818  0.1040  0.4905
 0.4824  0.4064  0.0770
 0.2465  0.2892  0.3712
[torch.FloatTensor of size 3x3]



In [14]:
(process_command(self, comm) == (x + y)).all()

True

In [15]:
(process_command(self, comm) == process_command(self, comm_m)).all()

True

In [16]:
(process_command(self, comm_cat) == torch.cat([x,y], dim=0)).all()

True

### Testing for Issue 133

In [17]:
from grid.clients.torch import TorchClient

/Users/jasonmancuso/anaconda/envs/openmined/lib/python3.6/site-packages/h5py-2.7.1-py3.6-macosx-10.7-x86_64.egg/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [18]:
# Must import torch before instantiating client,
# otherwise the hooked torch module will not be in the current working environment
import torch
from torch.autograd import Variable
known_workers=['/p2p-circuit/ipfs/QmQabt3SWuDvjse9z7GAcH2BGQv4wH8bumkd4x5oXN2obX']
client = TorchClient(verbose = False, include_github_known_workers=True, known_workers =  known_workers)


UPDATE: Connecting to IPFS... this can take a few seconds...

SUCCESS: Connected!!! - My ID: client:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4
all parts .... ['', 'Users', 'jasonmancuso', '.openmined']
full path /
full path /Users/
full path /Users/jasonmancuso/
full path /Users/jasonmancuso/.openmined/
Overloading Torch module
Overloading FloatTensor
Overloading DoubleTensor
Overloading HalfTensor
Overloading ByteTensor
Overloading CharTensor
Overloading ShortTensor
Overloading IntTensor
Overloading LongTensor
Overloading Variable
Overloading complete.


In [19]:
x = torch.FloatTensor(5)

In [20]:
x.id

6153933887

In [21]:
torch.add(x, x)

[['client:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4'], ['client:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4']]



 0.0000e+00
 4.0000e+00
-1.1197e+31
 3.1699e+29
 4.8628e-14
[torch.FloatTensor of size 5]

In [22]:
x.owners

['client:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4']

In [23]:
y = Variable(x)

In [24]:
y.owners

['client:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4']

In [25]:
torch.add(y, y)

[['client:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4'], ['client:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4']]


Variable containing:
 0.0000e+00
 4.0000e+00
-1.1197e+31
 3.1699e+29
 4.8628e-14
[torch.FloatTensor of size 5]

In [26]:
x.is_pointer = True
x.owners = ['worker:other_guy']

In [27]:
z1 = torch.add(x, x)

[['worker:other_guy'], ['worker:other_guy']]
Placeholder print for sending command to worker worker:other_guy
add
[<class 'torch.FloatTensor'>, <class 'torch.FloatTensor'>]
[]



In [28]:
z2 = x.add(x)

[['worker:other_guy']]
Placeholder print for sending command to worker worker:other_guy
add
[<class 'torch.FloatTensor'>]
[]



In [29]:
print(z1.id)
z1.owners

8773738021


['other_worker']

In [30]:
print(z2.id)
z2.owners

8881315526


['other_worker']

In [31]:
import json
import torch

In [32]:
tensor_type = torch.FloatTensor

In [33]:
self = torch.LongTensor(5)
self.id = 235412354
self.owners = ['a','b']

In [34]:
def ser(self, include_data=True):

    msg = {}
    msg['torch_type'] = self.type()
    if (include_data):
        msg['data'] = self.tolist()
    msg['id'] = self.id
    msg['owners'] = self.owners

    return json.dumps(msg)

In [35]:
str(torch.autograd.Variable)

"<class 'torch.autograd.variable.Variable'>"

In [36]:
torch.nn.parameter.Parameter(self)

Parameter containing:
 4.6117e+18
 4.6117e+18
 9.0000e+00
 1.6888e+15
 4.6117e+18
[torch.LongTensor of size 5]

In [37]:
msg = json.loads(ser(self))

In [38]:
map_torch_type = {
    'torch.FloatTensor':torch.FloatTensor,
    'torch.DoubleTensor':torch.DoubleTensor,
    'torch.HalfTensor':torch.HalfTensor,
    'torch.ByteTensor':torch.ByteTensor,
    'torch.CharTensor':torch.CharTensor,
    'torch.ShortTensor':torch.ShortTensor,
    'torch.IntTensor':torch.IntTensor,
    'torch.LongTensor':torch.LongTensor,
    'torch.autograd.variable.Variable':torch.autograd.variable.Variable,
    'torch.nn.parameter.Parameter':torch.nn.parameter.Parameter
}

In [39]:
def types_guard(obj_type):
    return map_torch_type[obj_type]

In [40]:
map_torch_type.values()

dict_values([<class 'torch.FloatTensor'>, <class 'torch.DoubleTensor'>, <class 'torch.HalfTensor'>, <class 'torch.ByteTensor'>, <class 'torch.CharTensor'>, <class 'torch.ShortTensor'>, <class 'torch.IntTensor'>, <class 'torch.LongTensor'>, <class 'torch.autograd.variable.Variable'>, <class 'torch.nn.parameter.Parameter'>])

In [41]:
print(type(msg['torch_type']))

<class 'str'>


In [42]:
types_guard(msg['torch_type'])

torch.LongTensor

In [43]:
def hook_tensor_serde(service_self, tensor_type):
    def ser(self, include_data=True):

        msg = {}
        msg['torch_type'] = tensor_type
        if (include_data):
            msg['data'] = self.tolist()
        msg['id'] = self.id
        msg['owner'] = self.owner

        return json.dumps(msg)

    def de(self, msg):
        if (type(msg) == str):
            msg = json.loads(msg)

        if ('data' in msg.keys()):
            v = torch.FloatTensor(msg['data'])
        else:
            v = torch.zeros(0)

        del service_self.objects[v.id]

        if (msg['id'] in service_self.objects.keys()):
            v_orig = service_self.objects[msg['id']].set_(v)
            return v_orig
        else:
            self.objects[msg['id']] = v
            v.id = msg['id']
            v.owner = msg['owner']
            return v

    tensor_type.ser = ser
    tensor_type.de = de

def hook_var_serde(service_self):
    # TODO
    pass

In [5]:
torch.add(x, x)

[['client:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4'], ['client:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4']]



 0.0000e+00
 7.3787e+19
 2.4978e+36
-7.3805e+19
-2.5272e-05
[torch.FloatTensor of size 5]

In [6]:
x.owners

['client:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4']

In [7]:
y = Variable(x)

In [8]:
y.owners

['client:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4']

In [9]:
torch.add(y, y)

[['client:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4'], ['client:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4']]


Variable containing:
 0.0000e+00
 7.3787e+19
 2.4978e+36
-7.3805e+19
-2.5272e-05
[torch.FloatTensor of size 5]

In [10]:
x.is_pointer = True
x.owners = ['worker:other_guy']

In [11]:
z1 = torch.add(x, x)

[['worker:other_guy'], ['worker:other_guy']]
Placeholder print for sending command to worker worker:other_guy
add
[<class 'torch.FloatTensor'>, <class 'torch.FloatTensor'>]
[]



In [12]:
z2 = x.add(x)

[['worker:other_guy'], ['worker:other_guy']]
Placeholder print for sending command to worker worker:other_guy
add
[<class 'torch.FloatTensor'>, <class 'torch.FloatTensor'>]
[]



In [13]:
print(z1.id)
z1.owners

5405206671


['other_worker']

In [14]:
print(z2.id)
z2.owners

9588806148


['other_worker']